# Implimenting abstractive text summerization

In [9]:
#import all the necessary libraries
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [40]:
#generate precize sentences
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    for i in range(len(filedata)):
        article.append(filedata[i].split(". "))
    #print(len(article[0]))    
    sentences = []
    for sentence in article[0]:
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" ")) 
    sentences.pop()
    return sentences

In [41]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [42]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

In [48]:
def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    # Step 1 - Read text and tokenize
    sentences =  read_article(file_name)
    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)
    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True) 
    for i in range(len(ranked_sentence)):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ". ".join(summarize_text))   
    print("\n\nFor indexes of top ranked_sentence order press x and then enter:\n\n")
    return ranked_sentence

In [49]:
x=generate_summary( "ab.txt", 5)

Summarize Text: 
 A VISUALLY STUNNING FILM TOPPED WITH GREAT PERFORMANCES STORY: A complex love story set during India’s Partition era, where the young Roop (Alia Bhatt) is torn between respect for husband Dev (Aditya Roy Kapur) and her newfound love for Zafar (Varun Dhawan). The other high points of the film are the problematic equation between father and son, Dev and Balraj as well as the fragile teacher-student dynamic between Bahaar Begum and Roop. While their back stories and heart-breaking love saga unfolds, the history of India takes an epic turn, from where there is no coming backREVIEW: As the name suggests, 'Kalank' is a story that questions the high morals of society, especially when it comes to love and family ties. In that aspect, writer and director Abhishek Varman's film makes a strong point.The story is based in a town called Husnabad, near Lahora few years before the Partition of India and Pakistan. And yet, there are sublime moments like the uneasy relationship betwee